## Import packages

In [245]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Hugh\\git\\warcraftLogs\\src.py'>

## Configurations

In [246]:
# Set path variables
path_to_adblockplus = r"C:\Users\Hugh\Desktop\3.11.2_0"
path_to_download_dir = r"C:\Users\Hugh\Downloads"

In [286]:
# Set character info
character_name = "Mercychann"
character_server = "Benediction"
character_region = "US"

In [285]:
# Number of parses to scrape
N_parses = 2
retry_attempts = 5

# Enable/disable SMS
twilio = True

## Navigate to character page

In [287]:
char_url, browser = src.load_individual_char_scraper(path_to_adblockplus, character_name, character_server, character_region)

In [288]:
bosses = src.get_t6_bosses(browser)
bosses

["High Warlord Naj'entus",
 'Supremus',
 'Shade of Akama',
 'Teron Gorefiend',
 'Gurtogg Bloodboil',
 'Reliquary of Souls',
 'Mother Shahraz',
 'The Illidari Council',
 'Illidan Stormrage',
 'Rage Winterchill',
 'Anetheron',
 "Kaz'rogal",
 'Azgalor',
 'Archimonde']

In [277]:
# Alternatively specify specific bosses
bosses = ['Archimonde']

In [250]:
# Dictionary for browser to navigate to boss page URLs
boss_link_dict = {"High Warlord Naj'entus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622"}

## Main code loop

In [289]:
def main():
    
    # Twilio is an SMS service that will send a text to your phone when the code finishes/crashes.
    # If you haven't setup twilio then set the flag to False in the Configuration cell.
    if twilio: 
        from twilio.rest import Client
        accountSID, authToken, myTwilioNumber, myCellPhone = src.get_twilio_info()
        twilioCli = Client(accountSID, authToken)
    
    
    retry_attempt = 0
    
    # The browser can crash at random due to the page loading too slowly. Rather than restarting the loop manually,
    # use 'retry_attempts' (default = 3) and the following while loop to restart automatically.
    while True: 
        
        try:
    
            for boss in bosses:

                browser.get(f"https://classic.warcraftlogs.com/character/{character_region.lower()}/{character_server.lower()}/{character_name.lower()}{boss_link_dict[boss]}")
                time.sleep(1)

                # Pick out the top parse of this boss
                search = browser.find_elements_by_class_name("character-table-link")
                boss_page_url = browser.current_url

                print(f"Boss: {boss}")

                #for i in range(len(search)):
                for i in range(4):

                    player_df = pd.DataFrame(pd.np.empty((0, 23)))
                    player_df.columns = ["Name", "Server", "Date", "Kill time", "Rank", "nHealers", "Spriest?", "Innervate?", "Bloodlust?", "Nature's Grace?", "Power Infusion?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotating on tank?", "Rotation 1", "% Rotation 1", "Rotation 2", "% Rotation 2"]

                    try:
                        rank, date, boss_rank, boss_HPS, boss_killtime, HPS_link = src.get_boss_data_char_scraper(browser, i)

                    except IndexError: 
                        continue

                    search = browser.find_elements_by_class_name("character-table-link")

                    if src.check_if_parse_already_recorded_char_scraper(i, browser, search, boss, character_name, character_server, character_region): 
                        print("Parse already recorded. Skipping")
                        browser.get(boss_page_url)
                        continue

                    HPS_link.click()
                    time.sleep(2)

                    temp_url = browser.current_url

                    boss_tanks = src.get_tanks(browser)
                    nHealers = src.get_nHealers(browser)

                    print(f"Tanks: {boss_tanks}")

                    browser.get(temp_url)
                    time.sleep(0.5)

                    player_link = browser.find_element_by_link_text(character_name)
                    player_link.click()
                    time.sleep(0.5)

                    # Scrape spell HPS
                    LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, boss_HPS)
                    time.sleep(2)

                    # Scrape mana sources
                    spriest = src.check_spriest(browser)
                    innervate, bloodlust, powerInfusion, naturesGrace = src.check_buffs(browser)

                    # Download the cast-sequence CSV.
                    src.download_csv(browser, temp_url, "filter-casts-tab", path_to_download_dir, "data/cast_sequence.csv")
                    time.sleep(1)

                    # Clean the csv
                    df = src.clean_cast_sequence_csv()
                    df = src.fix_cast_time(df)
                    time.sleep(0.5)

                    # Get the rotations
                    rotation1, rotation1_percent, rotation2, rotation2_percent, rotating_on_tank = src.calculate_rotations(df, boss, boss_tanks, LB_uptime)
                    print(f'Rotations: {rotation1} ({rotation1_percent}), {rotation2} ({rotation2_percent})')   

                    # Add data to pandas dataframe
                    to_append = [character_name, character_server + " " + character_region, date, boss_killtime, boss_rank, str(nHealers), spriest, innervate, bloodlust, powerInfusion, naturesGrace, LB_uptime, boss_HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotating_on_tank, rotation1, rotation1_percent, rotation2, rotation2_percent]
                    src.export_to_excel(boss, to_append, player_df, character_name, 'character_data', False)

                    os.remove(f"data/{boss.replace(' ', '')}_{character_name}.csv")
                    os.remove(f"data/cast_sequence.csv")
                    os.remove(f"data/cast_sequence_fixed.csv")

                    #Return to boss page
                    browser.get(boss_page_url)
                    time.sleep(1)  # Cannot be < 1s
                    print('------')

                print(f"{boss} scraping complete.")
                print("---------------")

             # Return to character page
            browser.get(char_url)
            
            message = twilioCli.messages.create(body='Data scraping complete!', from_=myTwilioNumber, to=myCellPhone)
            break
            
        except:
            retry_attempt += 1
            time.sleep(10)
            if retry_attempt == retry_attempts: 
                message = twilioCli.messages.create(body='Data scraping crashed', from_=myTwilioNumber, to=myCellPhone)
                break
    

if __name__ == "__main__":
    main()

Boss: High Warlord Naj'entus
Tanks: ['Felbear', 'Kipperrs', 'Klaren']
Boss: High Warlord Naj'entus
Tanks: ['Felbear', 'Kipperrs', 'Klaren']
Lifebloom uptime %: 100.00%
1LB 2I 1RG 0.19
1LB 3I 0RG 0.619
1LB 2I 0RG 0.143
3LB 1I 0RG 0.048
Non-tank rotations %: 0
Rotating on tank?: Yes
Rotations: 1LB 3I 0RG (0.619), 1LB 2I 1RG (0.19)
------
Tanks: ['Felbear', 'Kipperrs', 'Klaren']
Lifebloom uptime %: 99.87%
1LB 1I 2RG 0.579
1LB 2I 1RG 0.211
1LB 1I 1RG 0.211
Non-tank rotations %: 0
Rotating on tank?: Yes
Rotations: 1LB 1I 2RG (0.579), 1LB 2I 1RG (0.211)
------
Tanks: ['Felbear', 'Flashbot', 'Tanksiqt']
Lifebloom uptime %: 98.64%
1LB 0I 2RG 0.1
1LB 3I 0RG 0.4
1LB 2I 1RG 0.4
2LB 2I 0RG 0.05
1LB 1I 1RG 0.05
Non-tank rotations %: 0
Rotating on tank?: Yes
Rotations: 1LB 3I 0RG (0.4), 1LB 2I 1RG (0.4)
------
Tanks: ['Kipperrs', 'Klaren']
Lifebloom uptime %: 96.22%
2LB 2I 0RG 0.05
1LB 2I 0RG 0.2
1LB 2I 1RG 0.3
1LB 3I 0RG 0.3
1LB 1I 2RG 0.1
1LB 2I 2RG 0.05
Non-tank rotations %: 0
Rotating on tank?: 

------
Tanks: ['Felbear', 'Flashbot', 'Tanksiqt']
Lifebloom uptime %: 99.05%
1LB 2I 2RG 0.067
1LB 1I 1RG 0.2
1LB 2I 0RG 0.067
1LB 1I 2RG 0.033
1LB 0I 2RG 0.4
1LB 2I 1RG 0.133
2LB 1I 1RG 0.067
1LB 0I 3RG 0.033
Non-tank rotations %: 0
Rotating on tank?: Yes
Rotations: 1LB 0I 2RG (0.4), 1LB 1I 1RG (0.2)
------
Tanks: ['Felbear', 'Flashbot', 'Klaren']
Lifebloom uptime %: 97.95%
1LB 1I 1RG 0.25
1LB 2I 1RG 0.107
2LB 0I 2RG 0.036
1LB 1I 2RG 0.071
1LB 2I 0RG 0.143
1LB 0I 2RG 0.321
1LB 1I 3RG 0.036
1LB 3I 0RG 0.036
Non-tank rotations %: 0
Rotating on tank?: Yes
Rotations: 1LB 0I 2RG (0.321), 1LB 1I 1RG (0.25)
------
Mother Shahraz scraping complete.
---------------
Boss: The Illidari Council
Tanks: ['Felbear', 'Kipperrs', 'Klaren']
Lifebloom uptime %: 100.00%
3LB 0I 1RG 0.184
3LB 1I 0RG 0.316
2LB 1I 0RG 0.026
2LB 0I 2RG 0.079
2LB 2I 0RG 0.105
2LB 0I 1RG 0.026
2LB 1I 1RG 0.237
3LB 2I 0RG 0.026
Non-tank rotations %: 0
Rotating on tank?: Yes
Rotations: 3LB 1I 0RG (0.316), 2LB 1I 1RG (0.237)
------

Boss: Reliquary of Souls
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
Reliquary of Souls scraping complete.
---------------
Boss: Mother Shahraz
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
Mother Shahraz scraping complete.
---------------
Boss: The Illidari Council
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
The Illidari Council scraping complete.
---------------
Boss: Illidan Stormrage
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
Illidan Stormrage scraping complete.
---------------
Boss: Rage Winterchill
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
Parse already recorded. Skipping
Rage Wi

In [161]:
df = src.clean_cast_sequence_csv()
df = src.fix_cast_time(df)

In [164]:
df.head(10)

,Time,Type,Ability,Minute,Second,Cast Time,Target,Boss Target
0,00:00.305,Cast,生命之花,0,0.305,None,阿布的逆袭,NaN
1,00:01.705,Cast,生命之花,0,1.705,None,奶法利安,NaN
2,00:03.130,Cast,生命之花,0,3.130,None,Yikl,NaN
3,00:04.544,Cast,生命之花,0,4.544,None,战神丶哥哥,NaN
4,00:07.657,Cast,癒合,0,7.657,1.70,Vivicat,NaN
5,00:08.955,Cast,癒合,0,8.955,1.20,冷风袭屁屁,NaN
6,00:10.844,Cast,癒合,0,10.844,1.69,凯茜恬,NaN
7,00:12.075,Cast,癒合,0,12.075,1.23,冰礅礅,NaN
8,00:13.469,Cast,癒合,0,13.469,1.22,乱嘣嘣,NaN
9,00:14.888,Cast,癒合,0,14.888,1.22,梅迪尔丽,NaN


In [175]:
LB_uptime = '15.6%'
rotation1, rotation1_percent, rotation2, rotation2_percent, rotating_on_tank = src.calculate_rotations(df, boss, boss_tanks, LB_uptime)
print(f'Rotations: {rotation1} ({rotation1_percent}), {rotation2} ({rotation2_percent})')  

a
a
a
a
a
a
15.6%
0LB 4I 0RG 0.067
0LB 0I 4RG 0.867
0LB 1I 3RG 0.067
1.001
Rotating on tank?:  No
Rotations: 0LB 0I 4RG (0.867), 0LB 4I 0RG (0.067)


In [239]:
i=201

In [240]:
j = int(i/100.)

In [244]:
102%100

2